# Chapter 23 - RFormula

In [2]:
%run common.ipynb

Name: Error parsing magics!
Message: Magics [run] do not exist!
StackTrace: 

In [3]:
%ShowTypes on

Types will be printed.


# Setup

In [4]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row

import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame

import org.apache.spark.ml.feature._

import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

In [5]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<!-- To left align the HTML components in Markdown -->

### Spark parition control based on core availability

In [6]:
val NUM_CORES = 4
val NUM_PARTITIONS = 3

lazy val spark: SparkSession = SparkSession.builder()
    .appName("mllib-rformula")
    .getOrCreate()

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/
import spark.implicits._

NUM_CORES: Int = 4
NUM_PARTITIONS: Int = 3
spark: org.apache.spark.sql.SparkSession = <lazy>


spark: org.apache.spark.sql.SparkSession = <lazy>


In [7]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,192.168.0.27)
(spark.eventLog.enabled,true)
(spark.driver.port,34269)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://192.168.0.27:34269/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-1a5a1a8b-1f41-4e9a-9997-4bfdfc5bc496/repl-3f1802dd-fc0b-4dcd-be4e-d8457ed993c8)
(spark.app.name,mllib-rformula)
(spark.driver.memory,3g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,12)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://192.168.0.27:4041)
(spark.org.apache

configMap: Unit = ()


## Constants

In [8]:
val PROTOCOL="file://"
val DATA_DIR="/home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data"
val RESULT_DIR="."

PROTOCOL: String = file://
DATA_DIR: String = /home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data
RESULT_DIR: String = .


RESULT_DIR: String = .


# Tools

### Elapsed time profiler

In [9]:
import scala.collection.mutable.ListBuffer

val timing = new StringBuffer
val times = new ListBuffer[Long]()

def clear(): Unit = {
    timing.setLength(0)
    times.clear
}
def average(): Long = {
    times.reduce(_+_) / times.length
}

/**
@param label Description about the run
@code code to execute
@return execution
*/
def _timed[T](label: String, code: => T): T = {
    val start = System.currentTimeMillis()
    val result = code
    val stop = System.currentTimeMillis()
    timing.append(s"Processing $label took ${stop - start} ms.\n")
    times.append(stop - start)
    result
}

val timed = _timed _

timing: StringBuffer = 
times: scala.collection.mutable.ListBuffer[Long] = ListBuffer()
timed: (String, = > Nothing) => Nothing = <function2>


clear: ()Unit
average: ()Long
_timed: [T](label: String, code: => T)T


timed: (String, = > Nothing) => Nothing = <function2>


In [10]:
// To flush out error: missing argument list for method timed
println("")

<console>:59: error: missing argument list for method _timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `_timed _` or `_timed(_,_)` instead of `_timed`.
       _timed
       ^
lastException: Throwable = null


### Save to file

In [11]:
def save(df: DataFrame) = {
    df.coalesce(1)
    .write
    .format("csv")
    .mode(SaveMode.Overwrite)
    .option("header", "true")
    .save(RESULT_DIR)
}

save: (df: org.apache.spark.sql.DataFrame)Unit


# Dataframe

In [12]:
var df = spark.read.json(PROTOCOL + DATA_DIR + "/simple-ml")

df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


In [13]:
df.orderBy("value2").show(false)

+-----+----+------+------------------+
|color|lab |value1|value2            |
+-----+----+------+------------------+
|green|good|1     |14.386294994851129|
|green|bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|green|bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |bad |2     |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|green|good|1     |14.386294994851129|
|green|good|12    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|red  |good|35    |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



# RFormula

In [14]:
val supervised = new RFormula()
  .setFormula("lab ~ . + color:value1 + color:value2")

supervised: org.apache.spark.ml.feature.RFormula = RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_5ccacc76d850)


supervised: org.apache.spark.ml.feature.RFormula = RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_5ccacc76d850)


In [15]:
val fittedRF = supervised.fit(df)
val preparedDF = fittedRF.transform(df).orderBy("value2")
preparedDF.show(false)

+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|color|lab |value1|value2            |features                                                              |label|
+-----+----+------+------------------+----------------------------------------------------------------------+-----+
|green|good|1     |14.386294994851129|(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])  |1.0  |
|green|bad |16    |14.386294994851129|(10,[1,2,3,5,8],[1.0,16.0,14.386294994851129,16.0,14.386294994851129])|0.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])        |0.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])        |0.0  |
|blue |bad |12    |14.386294994851129|(10,[2,3,6,9],[12.0,14.386294994851129,12.0,14.386294994851129])      |0.0  |
|green|bad |16    |14.386294994851129|(10,[1,2,3,5,8],[1.0,16.0,14.38629

fittedRF: org.apache.spark.ml.feature.RFormulaModel = RFormulaModel(ResolvedRFormula(label=lab, terms=[color,value1,value2,{color,value1},{color,value2}], hasIntercept=true)) (uid=rFormula_5ccacc76d850)
preparedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 4 more fields]


preparedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 4 more fields]


# Split train/test data

In [16]:
val Array(train, test) = preparedDF.randomSplit(Array(0.7, 0.3))

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 4 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 4 more fields]


test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 4 more fields]


# Train a model

In [17]:
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features")
println(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial. (default: auto)
featuresCol: features column name (default: features, current: features)
fitIntercept: whether to fit an intercept term (default: true)
labelCol: label column name (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. (undefined)
maxIter: maximum number of iterations (>= 0) (default: 100)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for predicted clas

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_a0c79ac6ae27


lr: org.apache.spark.ml.classification.LogisticRegression = logreg_a0c79ac6ae27


## Prediction on test

In [18]:
val model = lr.fit(train)
model.transform(test).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_a0c79ac6ae27, numClasses = 2, numFeatures = 10


model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_a0c79ac6ae27, numClasses = 2, numFeatures = 10


## Model performance evaluation

In [19]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()
  .setMetricName("areaUnderROC")
  .setRawPredictionCol("prediction")
  .setLabelCol("label")

evaluator.evaluate(model.transform(test))

evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_73b49500ab5f


Double = 1.0
